Здесь надо привести нормализованные данные к входам дерева и обучиться на этих данных.

Для этого может потребоваться настройка параметров для фильтров, выбор фильтров, настройка агрегации и т.п, поэтому отдельно не выношу.